# Key Concepts

This section provides a brief introduction to some of the key concepts of the ``BrainState`` framework.

``BrainState`` is a high-performance computing framework dedicated to brain dynamics modelling, built on top of [JAX](https://github.com/jax-ml/jax). It provides neuroscience researchers, computational neuroscientists, and brain-like computing partitioner with a complete toolchain for building, optimising, and deploying neural network models of all kinds. It integrates modern hardware acceleration, automatic differentiation, event-driven computation, and other advanced features designed for neural networks, especially Spiking Neural Networks (SNNs). The following tutorials will introduce its core features and usage scenarios to help you get started and understand how to build and optimise brain dynamics models with BrainState. 